In [ ]:
import paho.mqtt.client as PahoMQTT

class MyMQTT:
	def __init__(self, clientID, broker, port, notifier):
		self.broker = broker
		self.port = port
		self.notifier = notifier
		self.clientID = clientID

		self._topic = ""
		self._isSubscriber = False

		# create an instance of paho.mqtt.client
		self._paho_mqtt = PahoMQTT.Client(clientID, False) 

		# register the callback
		self._paho_mqtt.on_connect = self.myOnConnect
		self._paho_mqtt.on_message = self.myOnMessageReceived


	def myOnConnect (self, paho_mqtt, userdata, flags, rc):
		print ("Connected to %s with result code: %d" % (self.broker, rc))

	def myOnMessageReceived (self, paho_mqtt , userdata, msg):
		# A new message is received
		self.notifier.notify (msg.topic, msg.payload)


	def myPublish (self, topic, msg):
		# if needed, you can do some computation or error-check before publishing
		print ("publishing '%s' with topic '%s'" % (msg, topic))
		# publish a message with a certain topic
		self._paho_mqtt.publish(topic, msg, 2)

	def mySubscribe (self, topic):
		# if needed, you can do some computation or error-check before subscribing
		print ("subscribing to %s" % (topic))
		# subscribe for a topic
		self._paho_mqtt.subscribe(topic, 2)

		# just to remember that it works also as a subscriber
		self._isSubscriber = True
		self._topic = topic

	def start(self):
		#manage connection to broker
		self._paho_mqtt.connect(self.broker , self.port)
		self._paho_mqtt.loop_start()

	def stop (self):
		if (self._isSubscriber):
			# remember to unsuscribe if it is working also as subscriber 
			self._paho_mqtt.unsubscribe(self._topic)

		self._paho_mqtt.loop_stop()
		self._paho_mqtt.disconnect()





In [ ]:


class DoSomething():
	def __init__(self, clientID):
		# create an instance of MyMQTT class
		self.clientID = clientID
		self.myMqttClient = MyMQTT(self.clientID, "test.mosquitto.org", 1883, self)
		


	def run(self):
		# if needed, perform some other actions befor starting the mqtt communication
		print ("running %s" % (self.clientID))
		self.myMqttClient.start()

	def end(self):
		# if needed, perform some other actions befor ending the software
		print ("ending %s" % (self.clientID))
		self.myMqttClient.stop ()

	def notify(self, topic, msg):
		# manage here your received message. You can perform some error-check here  
		print ("received '%s' under topic '%s'" % (msg, topic))






In [ ]:
from datetime import datetime


def Alert(type_alert, predicted, actual):
    now = datetime.now()
    dt_string = now.strftime("(%d/%m/%Y %H:%M:%S)")
    msg = {'datTime': dt_string, 'Quantity': type_alert, 'Predicted': predicted, 'Actual': actual}
    publisher = DoSomething("publisher 1")
    publisher.run()
    publisher.myMqttClient.myPublish('fdsfs', msg)
    publisher.end()


In [ ]:
import base64
import json
import cherrypy
from os.path import isfile, join
from os import listdir
import adafruit_dht
import tensorflow as tf
import numpy as np
import test.mosquitto.org as broker
import time




class AddModel: #DONE!
    exposed = True  # Needed for exposing the Web Services

    def POST(self,**query):
        if len(query)>0:
            raise cherrypy.HTTPError(400, 'Query detected! No query needed')
        body = cherrypy.request.body.read()
        body = json.loads(body)
        if len(body)>2:
            raise cherrypy.HTTPError(400, 'Body is longer than needed')
        if not body.get('name').endswith('.tflite'):
            raise cherrypy.HTTPError(400, 'The name extention should be tflite')
            
        model = body.get('model')
        model_name = body.get('name')
        decoded_model = base64.b64decode(model)
        path = './model/' + str(model_name)
        with open(path, 'wb') as f:
            f.write(decoded_model)
        


class ListModels: #DONE!
    exposed = True  # Needed for exposing the Web Services

    def GET(**query):
        if len(query)>0:
            raise cherrypy.HTTPError(400, 'Query detected! No query needed')
        models_path = './model'
        onlyFiles = [f for f in listdir(models_path) if isfile(join(models_path, f))]
        for i in onlyFiles:
            if i.endswith('.tflite'):
                print(i)


class Predict: #DONE!
    exposed = True  # Needed for exposing the Web Services
    def __init__(self):
        self.dht_device = adafruit_dht.DHT11(D4)        
    def PUT(self, **query):
        model_name = query.get('model')
        tthres = float(query.get('tthres'))
        hthres = float(query.get('hthres'))
        models_path = './model/' + model_name
        publisher = DoSomething("publisher 1")
        publisher.run()
        interpreter = tf.lite.Interpreter(model_path=models_path)
        interpreter.allocate_tensors()
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()
        temperature_list = []
        humidity_list = []
        for i in range(6):
            temperature_list.append(self.dht_device.temperature)
            humidity_list.append(self.dht_device.humidity)
            if i != 5:
                time.sleep(1)
        while True:
            input_data = np.array([temperature_list, humidity_list], dtype=np.float32)
            input_data = input_data.reshape(1, 6, 2)
            interpreter.set_tensor(input_details[0]['index'], input_data)
            interpreter.invoke()
            predict_result = interpreter.get_tensor(output_details[0]['index'])
            del temperature_list[0]
            del humidity_list[0]
            time.sleep(1)
            temperature_list.append(self.dht_device.temperature)
            humidity_list.append(self.dht_device.humidity)
            now = datetime.now()
            dt_string = now.strftime("(%d/%m/%Y %H:%M:%S)")
            if abs(predict_result[0][0] - temperature_list[5]) > tthres:
                msg = {'datTime': dt_string, 'Quantity': 'Temperature', 'Predicted': predict_result[0][0], 'Actual': temperature_list[5]}
                # Alert(type_alert='Temperature', predicted=10.1, actual=9)
            if abs(predict_result[0][1] - humidity_list[5]) > hthres:
                msg = {'datTime': dt_string, 'Quantity': 'Humidity', 'Predicted': predict_result[0][1], 'Actual': humidity_list[5]}
                # Alert(type_alert='Humidity', predicted=11, actual=15)
            publisher.myMqttClient.myPublish('aaa', json.dumps(msg))


if __name__ == '__main__':
    # conf probably needs modification
    conf = {
        '/': {
            'request.dispatch': cherrypy.dispatch.MethodDispatcher(),
            'tools.sessions.on': True,
        }
    }
    cherrypy.tree.mount(AddModel(), '/add', conf)
    cherrypy.tree.mount(ListModels, '/list', conf)
    cherrypy.tree.mount(Predict(), '/predict', conf)

    # To start cherrypy engine
    cherrypy.engine.start()
    cherrypy.engine.block()
